In [26]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle (1).json


In [27]:
!pwd

/content


In [25]:
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

cp: cannot stat '/content/.kaggle/kaggle.json': No such file or directory


In [28]:
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

100% 544M/544M [00:04<00:00, 131MB/s]

 99% 269M/271M [00:02<00:00, 117MB/s]
100% 271M/271M [00:02<00:00, 121MB/s]
  0% 0.00/111k [00:00<?, ?B/s]
100% 111k/111k [00:00<00:00, 105MB/s]


In [ ]:
import os, cv2, random, shutil
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
# Inspired by Jeff Delaney notebook: https://www.kaggle.com/jeffd23/catdognet-keras-convnet-starter

def get_images (directory, type=None):
    if type is None:
        images = [directory + i for i in os.listdir(directory)]
        print(f"There are {len(images)} images")
    else:
        images = [directory + i for i in os.listdir(directory) if type in i]
        print(f"There are {len(images)} {type} images")
    return images

base = 'drive/MyDrive/Masters/week08/data/'

train_images = get_images(base+'train')
train_dogs = get_images(base+'train', 'dog')
train_cats = get_images(base+'train', 'cat')
test_images = get_images('test')

There are 25000 images
There are 12500 dog images
There are 12500 cat images
There are 12500 images


In [ ]:
len(train_images)/6

4166.666666666667

In [ ]:
len(train_dogs)/6

2083.3333333333335

In [ ]:
train_images = random.sample(train_dogs, 2000) + random.sample(train_cats, 2000)
random.shuffle(train_images)

In [ ]:
hmm = train_images[1]
hmm[11:]


'cat.9799.jpg'

In [ ]:
for file in train_images:
    shutil.copy(file, f"data/train_subset/{file[11:]}")

In [ ]:
ROWS = 64
COLS = 64
CHANNELS = 3

In [ ]:
def read_image (file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)


array([[[ 17,  11,  22],
        [ 21,  17,  22],
        [ 22,  20,  26],
        ...,
        [154, 146, 147],
        [151, 146, 147],
        [154, 149, 150]],

       [[ 22,  16,  27],
        [ 19,  15,  21],
        [ 27,  25,  31],
        ...,
        [176, 168, 169],
        [123, 117, 118],
        [170, 167, 164]],

       [[ 20,  14,  25],
        [ 19,  15,  20],
        [ 18,  16,  22],
        ...,
        [157, 149, 150],
        [169, 163, 164],
        [153, 148, 149]],

       ...,

       [[178, 172, 174],
        [177, 170, 175],
        [163, 155, 162],
        ...,
        [189, 183, 184],
        [181, 175, 176],
        [170, 164, 165]],

       [[170, 165, 162],
        [148, 138, 149],
        [167, 159, 166],
        ...,
        [158, 152, 153],
        [177, 171, 172],
        [148, 142, 143]],

       [[155, 148, 151],
        [164, 157, 160],
        [182, 176, 177],
        ...,
        [162, 157, 156],
        [131, 124, 129],
        [168, 160, 167]]

In [ ]:
def read_image (file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)

def prep_data(images):
    count = len(images)
    data = np.ndarray((count, CHANNELS, ROWS, COLS), dtype=np.uint8)
    
    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image.T
        if i%250 == 0: print (f"Processed {i} of {count}")
    return data

train = prep_data(train_images)
print(f"Train shape {train.shape}")

Procesed 0 of 4000
Procesed 250 of 4000
Procesed 500 of 4000
Procesed 750 of 4000
Procesed 1000 of 4000
Procesed 1250 of 4000
Procesed 1500 of 4000
Procesed 1750 of 4000
Procesed 2000 of 4000
Procesed 2250 of 4000
Procesed 2500 of 4000
Procesed 2750 of 4000
Procesed 3000 of 4000
Procesed 3250 of 4000
Procesed 3500 of 4000
Procesed 3750 of 4000
Train shape (4000, 3, 64, 64)


In [ ]:
train

array([[[[250, 244, 241, ..., 248, 244, 247],
         [247, 227, 220, ..., 230, 238, 246],
         [246, 234, 220, ..., 221, 233, 250],
         ...,
         [248, 235, 214, ..., 212, 231, 245],
         [246, 232, 231, ..., 225, 228, 245],
         [247, 242, 242, ..., 243, 244, 247]],

        [[246, 245, 245, ..., 250, 246, 249],
         [249, 233, 228, ..., 236, 240, 248],
         [248, 240, 229, ..., 229, 237, 249],
         ...,
         [251, 242, 223, ..., 218, 237, 247],
         [248, 238, 239, ..., 228, 230, 247],
         [250, 247, 246, ..., 245, 246, 249]],

        [[246, 245, 244, ..., 250, 246, 249],
         [249, 232, 227, ..., 235, 240, 248],
         [248, 239, 227, ..., 229, 238, 251],
         ...,
         [249, 239, 220, ..., 217, 236, 247],
         [248, 237, 238, ..., 228, 230, 247],
         [250, 246, 247, ..., 245, 246, 249]]],


       [[[ 17,  22,  20, ..., 178, 170, 155],
         [ 21,  19,  19, ..., 177, 148, 164],
         [ 22,  27,  18, ..., 